In [1]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import pickle

open training set and save first example as json

In [2]:
df_credit = pd.read_csv("resources/german_credit_data.csv",index_col=0)

f = open("temp.json", "w")
df_credit[:1].to_json(f, orient='records')
f.close()

build preprocessing and modelling pipeline

In [3]:
integer_features = list(df_credit.columns[df_credit.dtypes == 'int64'])
integer_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = list(df_credit.columns[df_credit.dtypes == 'object'])
categorical_features.remove('Risk')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', integer_transformer, integer_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])

separate X and y then fit

In [4]:
X = df_credit.drop('Risk', axis=1)
y = df_credit['Risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbo...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

save pipeline to file

In [5]:
clf_pickle = open('filename.pkl', 'wb')
pickle.dump(clf, clf_pickle)
clf_pickle.close()

read back in to prove it works

In [6]:
clf2_pickle = open('filename.pkl', 'rb')
clf2 = pickle.load(clf2_pickle)

predict for the json we saved earlier

In [8]:
df_json = pd.read_json('temp.json')
clf2.predict_proba(df_json)

array([[0.06745673, 0.93254327]])